In [1]:
import sys
import re
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from random import random
import numpy as np
import pandas as pd
import csv
from xml.dom import minidom
import xml.etree.ElementTree as et
import os

In [2]:
train = open("line-train.XML", "r")

In [3]:
sense = []
for line in train:
    sense.append(line)

In [4]:
str1 = ''.join(sense).lower()
sense1 = []
pattern = re.compile(r'senseid="(.*)"')
matches = pattern.finditer(str1)
for match in matches:
    sense1.append(match.group(1))

In [5]:
fdist_sense1 = FreqDist(sense1)
fdist_sense1

FreqDist({'phone': 178, 'product': 196})

In [6]:
sense2 = []
pattern1 = re.compile(r'<head>(.*)</head>')
matches1 = pattern1.finditer(str1)
for match1 in matches1:
    sense2.append(match1.group(1))

In [7]:
feat = []
pattern2 = re.compile(r'<head>(.*)</head>(.*)')
matches2 = pattern2.finditer(str1)
for match2 in matches2:
    feat.append(match2.group(2))

In [8]:
x = []
for line in feat:
    x.append(re.sub(r'(<s>|<\/s>)','',line))

In [9]:
feat1 = []
pattern3 = re.compile(r'(.*)<head>(.*)</head>')
matches3 = pattern3.finditer(str1)
for match3 in matches3:
    feat1.append(match3.group(1))

#feat1 the part before the head -- list

In [10]:
x1 = []
for line in feat1:
    x1.append(re.sub(r'(<s>|<\/s>)','',line))
#x1
#after removing the <s> from feat1  

In [11]:
# for line in feat1:
#     pat = re.compile(r'(([^\s>]+) [\S]+ )$')
#     mat = pat.search(line)
#     x =mat.group(1)
#     print(x)
past2 = []
for line in x1:
    past2.append(line.split()[-10:])

In [12]:
# type(past2[1])
#past2
# past2 - words before the head in the context

In [13]:
# str2 = ''.join(feat1)
# past2 = []
# pattern4 = re.compile(r'<\/s><s>([^<]*)')
# matches4 = pattern4.finditer(str2)
# for match4 in matches4:
#     past2.append(match4.group(1))
fut2 = []
for line in feat:
    fut2.append(line.rstrip('</s>').split()[:10])

In [14]:
df = pd.DataFrame(
    {'sense': sense1,
     'target_word': sense2
    })


In [15]:

#sense2- line/lines
#sense1- actual sense 

In [16]:
combined = [x+y for x,y in zip(past2,fut2)]
#combined

In [17]:
#combined 
#combining prev and after

In [18]:
bof_phone = []
bof_product = []

for i in range(len(sense1)):
    if(sense1[i]=="phone"):
        bof_phone.append(combined[i])
    else:
        bof_product.append(combined[i])        

In [19]:
#bof_phone

In [21]:
temp =[]
for item in bof_phone:
    temp += item
bof_phone = temp

temp = []
for item in bof_product:
    temp += item
bof_product = temp


In [23]:
fdist_phone = FreqDist(bof_phone)
fdist_product = FreqDist(bof_product)
#fdist_phone

In [24]:
#freq of total words
total = bof_phone + bof_product
fdist_total = FreqDist(total)
#fdist_total

In [25]:
#########################




#testing





##########################

In [26]:
test = open("line-test.XML", "r")
#test

In [27]:
t_sense = []
for line in test:
    t_sense.append(line)

In [28]:
#t_sense

In [29]:
str2 = ''.join(t_sense).lower()
t_sense1 = []
pattern = re.compile(r'instance id="(.*)"')
matches = pattern.finditer(str2)
for match in matches:
    t_sense1.append(match.group(1))

In [30]:
len(t_sense1)

126

In [31]:
t_sense2 = []
pattern = re.compile(r'<head>(.*)</head>')
matches = pattern.finditer(str2)
for match in matches:
    t_sense2.append(match.group(1))

In [32]:
#t_sense2

In [33]:
t_feat = []
pattern = re.compile(r'<head>(.*)</head>(.*)')
matches = pattern.finditer(str2)
for match in matches:
    t_feat.append(match.group(2))

In [34]:
#t_feat

In [35]:
x = []
for line in t_feat:
    x.append(re.sub(r'(<s>|<\/s>)','',line))

In [36]:
t_feat1 = []
pattern = re.compile(r'(.*)<head>(.*)</head>')
matches = pattern.finditer(str2)
for match in matches:
    t_feat1.append(match.group(1))
#t_feat1

In [37]:
x1 = []
for line in t_feat1:
    x1.append(re.sub(r'(<s>|<\/s>)','',line))
#x1    

In [38]:
t_past2 = []
for line in x1:
    t_past2.append(line.split()[-10:])

In [39]:
#t_past2

In [40]:
t_fut2 = []
for line in x:
    t_fut2.append(line.rstrip('</s>').split()[:10])

In [41]:
#t_fut2

In [42]:
t_combined = [x+y for x,y in zip(t_past2,t_fut2)]
#t_combined

In [44]:
#probabilities P(s)
output = []
temp = 0
for j in range(len(t_combined)):
    for i in range(len(t_combined[j])):
        #sense phone
        prob_phone = []
        if t_combined[j][i] in total:
            prob_word = fdist_phone[t_combined[j][i]]/fdist_total[t_combined[j][i]]
            prob_feature = fdist_phone[t_combined[j][i]]/fdist_sense1["phone"]
            prob_phone.append(prob_word * prob_feature)
        else:
            prob_phone.append(fdist_sense1["phone"]/len(sense1))
            temp+=1

        #sense product
        prob_product = []
        if t_combined[j][i] in total:
            prob_word = fdist_product[t_combined[j][i]]/fdist_total[t_combined[j][i]]
            prob_feature = fdist_product[t_combined[j][i]]/fdist_sense1["product"]
            prob_product.append(prob_word * prob_feature)
        else:
            prob_product.append(fdist_sense1["product"]/len(sense1))
            temp+=1
        
    final_phone = 1
    final_product = 1
    
    for item in prob_phone:
        final_phone = final_phone * item
    for item in prob_product:
        final_product = final_product * item
    
    if(final_phone>final_product):
        output.append("phone")
    else:
        output.append("product")

In [45]:
#temp

In [46]:
answers = open("line-answers.txt", "r")
answers2 = []
for line in answers:
    answers2.append(line)
#answers2    

In [47]:
str3 = ''.join(answers2)
sense_ans = []
pattern = re.compile(r'senseid="(.*)"')
matches = pattern.finditer(str3)
for match in matches:
    sense_ans.append(match.group(1))

In [48]:
#accuracy

sum = 0
for i in range(len(output)):
    if(output[i]==sense_ans[i]):
        sum+=1
accuracy = sum/len(output)
accuracy

0.5158730158730159

In [ ]:
#intial accuracy = 51.58%